# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
import csv

# Import API key
from api_keys import g_key


### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
# Output File (CSV)
output_data_file = "./output_data/cities.csv"

# we could simply do this:
city_data_df = pd.read_csv(output_data_file)

# city_data = []

# with open(output_data_file, 'r', encoding='cp850') as f:
#     csv_obj = csv.reader(f)

#     header = next(csv_obj)

#     for row in csv_obj:
#         name = row[1]
#         lat = float(row[2])
#         lon = float(row[3])
#         max_temp = float(row[4])
#         humidity = float(row[5])
#         cloud = float(row[6])
#         wind_speed = float(row[7])
#         country = row[8]
#         date = int(row[9])
#         city = {'City': name, 'Lat': lat, 'Lon': lon, 'Max Temp': max_temp,
#                     'Humidity': humidity, 'Cloudiness': cloud, 'Wind Speed': wind_speed,
#                     'Country': country, 'Date': date}
#         city_data.append(city)
        
# city_data_df = pd.DataFrame(city_data)
print(city_data_df)
# !jupyter nbextension enable --py gmaps

     Unnamed: 0          City      Lat       Lon  Max Temp  Humidity  \
0             0       Rikitea -23.1203 -134.9692     69.04        57   
1             1  Puerto Ayora  -0.7393  -90.3518     78.76        82   
2             2          Uvat  59.1433   68.8953     57.27        90   
3             3   Ust'-Omchug  61.1500  149.6333     41.61        63   
4             4   Port Alfred -33.5906   26.8910     58.30        80   
..          ...           ...      ...       ...       ...       ...   
555         555        Bugiri   0.5714   33.7417     63.57        70   
556         556     Mikhaylov  54.2292   39.0186     65.48        84   
557         557        Urusha  54.0500  122.9000     52.70        98   
558         558      La Ronge  55.1001 -105.2842     86.25        25   
559         559       Xinzhou  38.4092  112.7333     68.13        85   

     Cloudiness  Wind Speed Country        Date  
0            77       20.09      PF  1627939125  
1            72       11.95      EC

### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [3]:
from ipywidgets.embed import embed_minimal_html

gmaps.configure(api_key=g_key)

humidity = city_data_df['Humidity']
loc = city_data_df[['Lat', 'Lon']]
humidity

figure = gmaps.figure()
heat = gmaps.heatmap_layer(locations=loc, weights=humidity, dissipating=False, max_intensity=300, point_radius=5)

figure.add_layer(heat)
embed_minimal_html('export0.html', views=[figure])
figure

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [4]:
# A max temperature lower than 80 degrees but higher than 70.
# Wind speed less than 10 mph.
# Zero cloudiness.
ideal_data_df = city_data_df[(city_data_df['Max Temp'] > 70) & (city_data_df['Max Temp'] < 80)
                         & (city_data_df['Wind Speed'] < 10) & (city_data_df['Cloudiness'] == 0)]

ideal_data_df
# Drop any rows that don’t contain all three conditions. You want to be sure the weather is ideal.
ideal_data_df.dropna()

,Unnamed: 0,City,Lat,Lon,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
29,29,San Juan,-31.5375,-68.5364,71.87,16,0,5.99,AR,1627939394
91,91,Vila Velha,-20.3297,-40.2925,73.96,77,0,1.14,BR,1627939003
175,175,Balyqshy,47.0667,51.8667,78.80,22,0,6.71,KZ,1627939437
222,222,Kalemie,-5.9475,29.1947,70.56,57,0,1.72,CD,1627939452
298,298,Birjand,32.8663,59.2211,70.27,17,0,9.22,IR,1627939475
301,301,Seoul,37.5683,126.9778,76.51,100,0,4.61,KR,1627939166
335,335,Vila Franca do Campo,37.7167,-25.4333,70.21,78,0,7.14,PT,1627939487
402,402,Juegang,32.3174,121.1855,77.65,93,0,9.55,CN,1627939509
412,412,San Lorenzo,-25.3333,-57.5333,70.74,37,0,8.05,PY,1627939513
415,415,Hede,33.7681,120.2517,74.28,91,0,5.50,CN,1627939513


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [ ]:
hotel_df = ideal_data_df[['City', 'Country', 'Lat', 'Lon']].copy()

hotel_df['Hotel Name'] = ""
hotel_df

In [ ]:
params = {
    'radius': 5000,
    'types': 'lodging',
    'key': g_key
}
temp = []
for index, row in hotel_df.iterrows():
    lat = row['Lat']
    lon = row['Lon']
    
    params['location'] = f'{lat}, {lon}'

    url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    addr = requests.get(url, params=params).json()
    try:
        hotel_df.loc[index, 'Hotel Name'] = addr['results'][0]['name']
        temp.append(addr)
    except Exception as e:
        print('Missing info')
        
hotel_df

In [ ]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
try:
    hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
    locations = hotel_df[["Lat", "Lon"]]
except Exception as e:
    print(e)


In [ ]:
# Add marker layer ontop of heat map
from ipywidgets.embed import embed_minimal_html
marker = gmaps.marker_layer(locations, info_box_content=hotel_info)
figure.add_layer(marker)
# Display figure
#plt.savefig(f"./pngs/LATvs{y_label}.png")
embed_minimal_html('export.html', views=[figure])
figure